In [10]:
import cvzone
from cvzone.HandTrackingModule import HandDetector
import cv2 as cv
import math
import serial
import time

In [11]:
ser=serial.Serial('COM3',9600,timeout=1)

In [13]:
cap=cv.VideoCapture(0)
detector=HandDetector(detectionCon=0.8,maxHands=1)
on_normal=True

while True:
    sucess,img=cap.read()
    img=cv.flip(img,1)
    hands,img = detector.findHands(img)
    if hands:
        hands1=hands[0]
        lmlist1=hands1['lmList']
        bbox1=hands1["bbox"]
        
        if lmlist1:
            fingers=detector.fingersUp(hands1)
            #print(fingers)
            #fingers detection
            try:
                if fingers==[0,1,1,0,0] or fingers==  [1,1,1,0,0]:
                    #print('gripper close',fingers)
                    ser.write(b'f')
                if fingers==[1,1,1,1,0] or fingers== [0,1,1,1,0]:
                   # print('gripper open',fingers)
                    ser.write(b'e')
                time.sleep(0.03)
            except:
                print("error in thumb")
        
            if fingers[4]==1:
                
                xt,yt=lmlist1[4][0],lmlist1[4][1]
                xi,yi=lmlist1[8][0],lmlist1[8][1]
                xib,yib=lmlist1[5][0],lmlist1[5][1]
                xpb,ypb=lmlist1[17][0],lmlist1[17][1]
                x0,y0=lmlist1[0][0],lmlist1[0][1]
                
                #print(xt,yt)
                #print(lmlist1)
                cv.circle(img,(xt,yt),5,(255,0,255),cv.FILLED)
                cv.circle(img,(xi,yi),5,(255,0,255),cv.FILLED)
                cv.circle(img,(xib,yib),5,(255,0,255),cv.FILLED)
                cv.circle(img,(xpb,ypb),5,(255,0,255),cv.FILLED)
                cv.circle(img,(x0,y0),5,(255,0,255),cv.FILLED)
                cv.line(img,(x0,y0),(xi,yi),(255,0,255),2)
                
                #finding length
                length_thumb=math.hypot(xt-x0,yt-y0)
                length_roatate=math.hypot(xib-xpb,yib-ypb)
                length_index=math.hypot(xi-x0,yi-y0)
                #print(length_roatate)
                try:
                    if length_thumb <70:
                       # print("decrease height",length_thumb)
                        cv.circle(img,(xt,yt),5,(255,0,0),cv.FILLED)
                        ser.write(b'g')
                    if length_thumb > 105:
                       # print('increase height',length_thumb)
                        cv.circle(img,(xt,yt),5,(255,0,0),cv.FILLED)
                        ser.write(b'h')
                    if length_index<80:
                       # print("contrcat")
                        cv.circle(img,(xi,yi),5,(255,0,0),cv.FILLED)
                        ser.write(b'd')
                    if length_index>150:
                       # print('expand')
                        cv.circle(img,(xi,yi),5,(255,0,0),cv.FILLED)
                        ser.write(b'c')
                    if length_roatate<30:
                        ser.write(b'j')
                        on_normal=False
                        cv.circle(img,(xpb,ypb),5,(255,0,0),cv.FILLED)
                    if on_normal is False: 
                        if length_roatate>60:
                            on_normal=True
                            ser.write(b'k')
                            cv.circle(img,(xpb,ypb),5,(255,0,0),cv.FILLED)
                        
                except:
                    print('error occured')

            
    cv.imshow("hand detection",img)
    key=cv.waitKey(1)
    if key==ord('q'):
        cap.release()
        break
cv.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [ ]:
cap